<a href="https://colab.research.google.com/github/JunghyeonAhn/SQL-Project-/blob/main/Streaming_correlation_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import data

In [1]:
import pandas as pd

In [2]:
# load data
df = pd.read_csv('/content/spotify_202511041529.csv')

In [3]:
print(df.columns)

Index(['track_name', 'artist(s)_name', 'artist_count', 'released_year',
       'released_month', 'released_day', 'in_spotify_playlists',
       'in_spotify_charts', 'streams', 'in_apple_playlists', 'in_apple_charts',
       'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts', 'bpm',
       'key', 'mode', 'danceability_%', 'valence_%', 'energy_%',
       'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%'],
      dtype='object')


In [4]:
# --- 2. Robust Data Cleaning (콤마/문자열 오류 해결) ---

# 대상 칼럼: streams와 모든 playlist/charts 칼럼
numeric_cols_to_clean = [
    'streams',
    'in_spotify_playlists', 'in_apple_playlists', 'in_deezer_playlists', 'in_shazam_charts'
]

for col in numeric_cols_to_clean:
    # 콤마 및 비숫자 문자 제거 (ValueError의 원인인 '3,421'과 같은 콤마 처리)
    if df[col].dtype == 'object':
        df[col] = df[col].astype(str).str.replace(r'[^\d.]', '', regex=True)

    # 안전하게 숫자형으로 변환 (오류 시 NaN으로 처리)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 분석에 필요한 모든 칼럼에서 NaN이 있는 행 제거 (acousticness_% 포함)
df.dropna(subset=numeric_cols_to_clean + ['acousticness_%'], inplace=True)

## Aggregatin data by tracks and Artists

In [5]:
# aggregation by tracks
agg_dict ={
    'streams': 'sum',
    'in_spotify_playlists': 'max',
    'in_apple_playlists': 'max',
    'acousticness_%': 'mean',
    'in_deezer_playlists': 'max',
    'in_shazam_charts': 'max',
}
df_aggregated = df.groupby(['track_name', 'artist(s)_name']).agg(agg_dict).reset_index()

In [6]:
# Naming coulumn,
df_aggregated.rename(columns={
    'streams':'total_streams',
    'in_spotify_playlists':'total_spotify_playlists',
    'in_apple_playlists':'total_apple_playlists',
    'acousticness_%':'avg_acousticness',
    'in_deezer_playlists' :'total_deezer_playlists',
    'in_shazam_charts':'in_shazam_charts'
},inplace=True)

df_top50 = df_aggregated.nlargest(50,'total_streams')

In [7]:
# defining analysing target
df_corr = df_top50[['total_streams', 'total_spotify_playlists', 'total_apple_playlists','total_deezer_playlists','in_shazam_charts', 'avg_acousticness']]

## Correlation Analysis

### Correlation Analysis 01. Is 'Acousticness' success factor in song market?

In [8]:
from numpy._core import numeric
#Calculate correlation
df_corr = df_top50[['total_streams', 'total_spotify_playlists', 'total_apple_playlists','total_deezer_playlists','in_shazam_charts', 'avg_acousticness']]

correlation_matrix = df_corr.corr()

# 1
print("Correlation of Acousticness:")
print(correlation_matrix.loc[['avg_acousticness'],['total_streams','total_spotify_playlists']])

Correlation of Acousticness:
                  total_streams  total_spotify_playlists
avg_acousticness       0.086931                -0.327755


### Correlation Analysis 02. Is there any correlation between platforms? - Spotify vs Apple


In [9]:
# 2
print("Correlation by platform(Spotify vs. Apple):")
platform_corr = correlation_matrix.loc['total_spotify_playlists', 'total_apple_playlists']
print(f"{platform_corr:.4f}")

Correlation by platform(Spotify vs. Apple):
0.2564


### Correlation Analysis 03. Is there any correlation between platforms? - Spotify vs Deezer


In [10]:
print("Correlation by platform(Spotify vs. Deezer):")
platform_corr = correlation_matrix.loc['total_spotify_playlists', 'total_deezer_playlists']
print(f"{platform_corr:.4f}")

Correlation by platform(Spotify vs. Deezer):
0.6194


### Correlation Analysis 04. Correlation: Shazam Charts VS Total Streams


In [14]:
print("Correlation: Shazam Charts VS Total Streams ")
shazam_stream_corr = correlation_matrix.loc['in_shazam_charts', 'total_streams']
print(f"{shazam_stream_corr:.4f}")

Correlation: Shazam Charts VS Total Streams 
-0.1693
